# wrds_crspm

> Retrieve and process data from WRDS CRSP Monthly Stock File

Since this is a proprietary dataset, in the documentation below, I can not show any of the data that is being retrieved/generated (will show only column names).

In [ ]:
#| default_exp wrds.crspm

In [ ]:
#|exports
from __future__ import annotations
from pathlib import Path
from typing import List
import os

import pandas as pd
import numpy as np

import pandasmore as pdm
from finsets.wrds import wrds_api
from finsets import RESOURCES

In [ ]:
#| export 
def raw_metadata(rawfile: str|Path=RESOURCES/'crspm_variable_descriptions.csv', # location of the raw variable labels file
             ) -> pd.DataFrame:
    "Loads raw variable labels file, cleans it and returns it as a pd.DataFrame"

    df = pd.read_csv(rawfile)
    df['output_of'] = 'wrds.crspm.clean'

    df['Variable Label'] = df.apply(lambda row: row['Description'].replace(row['Variable Name'].strip()+' -- ', ''), axis=1)
    df['Variable Label'] = df.apply(lambda row: row['Variable Label'].replace( '(' + row['Variable Name'].strip() + ')', ''), axis=1)
    df['Variable Name'] = df['Variable Name'].str.strip().str.lower()
    df = df[['Variable Name', 'Variable Label','output_of', 'Type']].copy()
    df.columns = ['name','label','output_of','type']
    return df

In [ ]:
raw_metadata()

,name,label,output_of,type
0,cusip,Cusip,wrds.crspm.clean,string
1,ncusip,Ncusip,wrds.crspm.clean,string
2,comnam,Company Name,wrds.crspm.clean,string
3,ticker,Ticker,wrds.crspm.clean,string
4,permco,CRSP Permanent Company Number,wrds.crspm.clean,double
...,...,...,...,...
57,vwretd,Value-Weighted Return (includes distributions),wrds.crspm.clean,double
58,vwretx,Value-Weighted Return (excluding dividends),wrds.crspm.clean,double
59,ewretd,Equal-Weighted Return (includes distributions),wrds.crspm.clean,double
60,ewretx,Equal-Weighted Return (excluding dividends),wrds.crspm.clean,double


The following function gives more detailed metadata but requires connecting to WRDS. If all you want is variable names and labels, then `raw_metadata` is sufficient.

In [ ]:
#| export
def raw_metadata_extra(wrds_username: str=None
             ) -> pd.DataFrame:
    "Collects metadata from WRDS `crsp.msf` and `crsp.msenames` tables and merges it with `variable_labels`."

    if wrds_username is None:
        wrds_username = os.getenv('WRDS_USERNAME')
        if wrds_username is None: wrds_username = input("Enter your WRDS username: ") 

    try:
        db = wrds_api.Connection(wrds_username = wrds_username)
        msf = db.describe_table('crsp','msf')
        msf_rows = db.get_row_count('crsp','msf')
        mse = db.describe_table('crsp','msenames')
        mse_rows = db.get_row_count('crsp','msenames')
    finally:
        db.close()
        
    msf_meta = msf[['name','type']].copy()
    msf_meta['nr_rows'] = msf_rows
    msf_meta['wrds_library'] = 'crsp'
    msf_meta['wrds_table'] = 'msf'

    mse_meta = mse[['name','type']].copy()
    mse_meta['nr_rows'] = mse_rows
    mse_meta['wrds_library'] = 'crsp'
    mse_meta['wrds_table'] = 'msenames'

    crsp_meta = (pd.concat([msf_meta, mse_meta],axis=0, ignore_index=True)
                .merge(raw_metadata()[['name','label']], how='left', on='name'))
    
    crsp_meta['output_of'] = 'wrds.crspm.download()'
    crsp_meta = pdm.order_columns(crsp_meta,these_first=['name','label','output_of'])
    for v in list(crsp_meta.columns):
        crsp_meta[v] = crsp_meta[v].astype('string')
    
    return crsp_meta

In [ ]:
#| eval: false
raw_metadata_extra()

Loading library list...
Done
Approximately 4922867 rows in crsp.msf.
Approximately 111623 rows in crsp.msenames.


,name,label,output_of,type,nr_rows,wrds_library,wrds_table
0,cusip,Cusip,wrds.crspm.download(),VARCHAR(8),4922867,crsp,msf
1,permno,<NA>,wrds.crspm.download(),DOUBLE_PRECISION,4922867,crsp,msf
2,permco,CRSP Permanent Company Number,wrds.crspm.download(),DOUBLE_PRECISION,4922867,crsp,msf
3,issuno,Nasdaq Issue Number,wrds.crspm.download(),DOUBLE_PRECISION,4922867,crsp,msf
4,hexcd,Header Exchange Code,wrds.crspm.download(),DOUBLE_PRECISION,4922867,crsp,msf
5,hsiccd,Header SIC Code,wrds.crspm.download(),DOUBLE_PRECISION,4922867,crsp,msf
6,date,<NA>,wrds.crspm.download(),DATE,4922867,crsp,msf
7,bidlo,Bid or Low,wrds.crspm.download(),DOUBLE_PRECISION,4922867,crsp,msf
8,askhi,Ask or High,wrds.crspm.download(),DOUBLE_PRECISION,4922867,crsp,msf
9,prc,Price,wrds.crspm.download(),DOUBLE_PRECISION,4922867,crsp,msf


In [ ]:
#| export
def default_raw_vars():
    """Default variables used in `download` if none are specified. Takes about 2 min to download."""
    
    return ['permno','permco','date',
            'ret', 'retx', 'shrout', 'prc', 
            'shrcd', 'exchcd','siccd','ticker','cusip','ncusip']            

In [ ]:
print(default_raw_vars())

['permno', 'permco', 'date', 'ret', 'retx', 'shrout', 'prc', 'shrcd', 'exchcd', 'siccd', 'ticker', 'cusip', 'ncusip']


In [ ]:
#| export
def parse_varlist(vars: List[str]=None,
                  wrds_username: str=None
                  ) -> str:
    """Figure out which `vars` come from the `crsp.msf` table and which come from the `crsp.msenames` table and add a. and b. prefixes"""

    if wrds_username is None:
        wrds_username = os.getenv('WRDS_USERNAME')
        if wrds_username is None: wrds_username = input("Enter your WRDS username: ") 

    if vars is None: vars = default_raw_vars()
    vars = ['permno','permco','date','exchcd'] + [x for x in vars if x not in ['permno','permco','date','exchcd']]

    try:
        db = wrds_api.Connection(wrds_username = wrds_username)
        all_msf_vars = list(db.describe_table('crsp','msf').name)
        all_mse_vars = list(db.describe_table('crsp','msenames').name)
        my_msf_vars = [f'a.{x}' for x in vars if x in all_msf_vars]
        my_mse_vars = [f'b.{x}' for x in vars if (x in all_mse_vars) and (x not in all_msf_vars)]
        varlist_string = ','.join(my_msf_vars + my_mse_vars)
    finally:
        db.close()
        
    return varlist_string

In [ ]:
#| export
def delist_adj_ret(df: pd.DataFrame, # Requires `ret`,`exchcd`, ` `dlret`, and `dlstcd` variables
                       adj_ret_var: str
                       ) -> pd.DataFrame:
    """Adjust for delisting returns using Shumway and Warther (1999) and Johnson and Zhao (2007)"""

    df['npdelist'] = (df['dlstcd']==500) | df['dlstcd'].between(520,584)
    df['dlret'] = np.where(df.dlret.isna() & df.npdelist & df.exchcd.isin([1,2]), -0.35, df.dlret)
    df['dlret'] = np.where(df.dlret.isna() & df.npdelist & df.exchcd.isin([3]), -0.55, df.dlret)
    df['dlret'] = np.where(df.dlret.notna() & df.dlret < -1, -1, df.dlret)
    df['dlret'] = df.dlret.fillna(0)

    df[adj_ret_var] = (1 + df.ret) * (1 + df.dlret) - 1
    df[adj_ret_var] = np.where(df[adj_ret_var].isna() & (df.dlret!=0), df.dlret, df[adj_ret_var])
    df = df.drop('npdelist', axis=1) 
    return df

In [ ]:
#| export
def download(vars: List[str]=None, # If None, downloads `default_raw_vars`; `permno`, `permco`, `date`, and 'exchcd' are added by default
             obs_limit: int=None,  #Number of rows to download. If None, full dataset will be downloaded             
             wrds_username: str=None,       #If None, looks for WRDS_USERNAME with `os.getenv`, then prompts you if needed
             start_date: str="01/01/1900",  # Start date in MM/DD/YYYY format
             end_date: str=None,            # End date in MM/DD/YYYY format; if None, defaults to current date  
             add_delist_adj_ret: bool=True, # Whether to calculate delisting-adjusted returns 
             adj_ret_var: str='ret_adj'     # What to call the returns adjusted for delisting bias
             ) -> pd.DataFrame:
    """Downloads `vars` from `start_date` to `end_date` from WRDS crsp.msf and crsp.msenames libraries. 
        Creates `ret_adj` for delisting based on Shumway and Warther (1999) and Johnson and Zhao (2007)"""

    varlist_string = parse_varlist(vars, wrds_username)
    limit_clause = f"LIMIT {obs_limit}" if obs_limit is not None else ""
    sql_string = f"""SELECT {varlist_string},  c.dlstcd, c.dlret 
                        FROM crsp.msf AS a 
                        LEFT JOIN crsp.msenames AS b
                            ON a.permno=b.permno AND b.namedt<=a.date AND a.date<=b.nameendt                     
                        LEFT JOIN crsp.msedelist as c
                            ON a.permno=c.permno AND date_trunc('month', a.date) = date_trunc('month', c.dlstdt)                            
                            WHERE a.date BETWEEN '{start_date}' AND COALESCE(%(end)s, CURRENT_DATE) 
                    {limit_clause}
                """
    df = wrds_api.download(sql_string, wrds_username=wrds_username, params={'end':end_date})
    if add_delist_adj_ret: df = delist_adj_ret(df, adj_ret_var)
    else: df = df.drop(['dlret','dlstcd'], axis=1)
    return df 

In [ ]:
#| eval: false
raw = download(start_date='01/01/2021', obs_limit=100)

Loading library list...
Done
Approximately 4922867 rows in crsp.msf.
Approximately 111623 rows in crsp.msenames.
Loading library list...
Done


In [ ]:
#| eval: false
raw

,permno,permco,date,ret,retx,shrout,prc,cusip,exchcd,shrcd,siccd,ticker,ncusip,dlstcd,dlret,ret_adj
0,10026.0,7976.0,2021-01-29,-0.017442,-0.017442,18980.0,152.660004,46603210,3.0,11.0,2052.0,JJSF,46603210,None,0,-0.017442
1,10028.0,7978.0,2021-01-29,0.203846,0.203846,26925.0,6.260000,29402E10,2.0,11.0,5094.0,ELA,29402E10,None,0,0.203846
2,10032.0,7980.0,2021-01-29,-0.016494,-0.016494,28766.0,76.919998,72913210,3.0,11.0,3670.0,PLXS,72913210,None,0,-0.016494
3,10044.0,7992.0,2021-01-29,0.071605,0.071605,6074.0,4.340000,77467X10,3.0,11.0,2060.0,RMCF,77467X10,None,0,0.071605
4,10051.0,7999.0,2021-01-29,-0.068213,-0.068213,38109.0,20.490000,41043F20,1.0,11.0,4813.0,HNGR,41043F20,None,0,-0.068213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,11533.0,9360.0,2021-01-29,-0.119227,-0.119227,29236.0,450.109985,30325010,1.0,11.0,7372.0,FICO,30325010,None,0,-0.119227
96,11547.0,9370.0,2021-01-29,-0.000893,-0.000893,28602.0,111.900002,20741010,1.0,11.0,3841.0,CNMD,20741010,None,0,-0.000893
97,11565.0,9389.0,2021-01-29,0.005675,-0.001892,43881.0,15.830000,44040710,3.0,11.0,6021.0,HBNC,44040710,None,0,0.005675
98,11581.0,323.0,2021-01-29,-0.103494,-0.103494,59871.0,20.270000,29276510,1.0,11.0,3559.0,EPAC,29276510,None,0,-0.103494


In [ ]:
#| export
def clean(df: pd.DataFrame=None,        # If None, downloads `vars` using `download` function; else, must contain `permno` and `date` columns
          vars: List[str]=None,         # If None, downloads `default_raw_vars`
          obs_limit: int=None, #Number of rows to download. If None, full dataset will be downloaded
          wrds_username: str=None,      # If None, looks for WRDS_USERNAME with `os.getenv`, then prompts you if needed
          start_date: str="01/01/1900", # Start date in MM/DD/YYYY format
          end_date: str=None,           # End date. Default is current date          
          clean_kwargs: dict={},        # Params to pass to `pdm.setup_panel` other than `panel_ids`, `time_var`, and `freq`
          ) -> pd.DataFrame:
    """Applies `pandasmore.setup_panel` to `df`. If `df` is None, downloads `vars` using `download` function."""

    if df is None: df = download(vars=vars, obs_limit=obs_limit, wrds_username=wrds_username, start_date=start_date, end_date=end_date)
    df = pdm.setup_panel(df, panel_ids='permno', time_var='date', freq='M', **clean_kwargs)
    return df 

In [ ]:
#| eval: false
df = clean(df=raw)

In [ ]:
#| eval: false
df

,,date,dtdate,permco,ret,retx,shrout,prc,cusip,exchcd,shrcd,siccd,ticker,ncusip,dlstcd,dlret,ret_adj
permno,Mdate,,,,,,,,,,,,,,,,
10026,2021-01,2021-01-29,2021-01-29,7976.0,-0.017442,-0.017442,18980.0,152.660004,46603210,3.0,11.0,2052.0,JJSF,46603210,None,0,-0.017442
10028,2021-01,2021-01-29,2021-01-29,7978.0,0.203846,0.203846,26925.0,6.260000,29402E10,2.0,11.0,5094.0,ELA,29402E10,None,0,0.203846
10032,2021-01,2021-01-29,2021-01-29,7980.0,-0.016494,-0.016494,28766.0,76.919998,72913210,3.0,11.0,3670.0,PLXS,72913210,None,0,-0.016494
10044,2021-01,2021-01-29,2021-01-29,7992.0,0.071605,0.071605,6074.0,4.340000,77467X10,3.0,11.0,2060.0,RMCF,77467X10,None,0,0.071605
10051,2021-01,2021-01-29,2021-01-29,7999.0,-0.068213,-0.068213,38109.0,20.490000,41043F20,1.0,11.0,4813.0,HNGR,41043F20,None,0,-0.068213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11533,2021-01,2021-01-29,2021-01-29,9360.0,-0.119227,-0.119227,29236.0,450.109985,30325010,1.0,11.0,7372.0,FICO,30325010,None,0,-0.119227
11547,2021-01,2021-01-29,2021-01-29,9370.0,-0.000893,-0.000893,28602.0,111.900002,20741010,1.0,11.0,3841.0,CNMD,20741010,None,0,-0.000893
11565,2021-01,2021-01-29,2021-01-29,9389.0,0.005675,-0.001892,43881.0,15.830000,44040710,3.0,11.0,6021.0,HBNC,44040710,None,0,0.005675


We can download a small sample of the dataset and clean it in one step:

In [ ]:
#| eval: false
df = clean(obs_limit=100, vars=['ret','shrcd','exchcd'], start_date='01/01/2020', end_date='12/31/2020')

Loading library list...
Done
Approximately 4922867 rows in crsp.msf.
Approximately 111623 rows in crsp.msenames.
Loading library list...
Done


In [ ]:
#| eval: false
df

,,date,dtdate,permco,ret,exchcd,shrcd,dlstcd,dlret,ret_adj
permno,Mdate,,,,,,,,,
10026,2020-01,2020-01-31,2020-01-31,7976.0,-0.100016,3.0,11.0,None,0,-0.100016
10028,2020-01,2020-01-31,2020-01-31,7978.0,0.607407,2.0,11.0,None,0,0.607407
10032,2020-01,2020-01-31,2020-01-31,7980.0,-0.075643,3.0,11.0,None,0,-0.075643
10044,2020-01,2020-01-31,2020-01-31,7992.0,-0.098592,3.0,11.0,None,0,-0.098592
10051,2020-01,2020-01-31,2020-01-31,7999.0,-0.115176,1.0,11.0,None,0,-0.115176
...,...,...,...,...,...,...,...,...,...,...
11407,2020-01,2020-01-31,2020-01-31,53120.0,0.050475,4.0,73.0,None,0,0.050475
11441,2020-01,2020-01-31,2020-01-31,9278.0,0.152941,3.0,11.0,None,0,0.152941
11442,2020-01,2020-01-31,2020-01-31,9280.0,-0.069555,3.0,11.0,None,0,-0.069555


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()